In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.4.1
torchvision version: 0.19.1


In [2]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (142/142), done.
Receiving objects: 100% (4356/4356), 654.51 MiB | 10.27 MiB/s, done.
remote: Total 4356 (delta 213), reused 254 (delta 178), pack-reused 4035 (from 1)
Resolving deltas: 100% (2585/2585), done.
Updating files: 100% (248/248), done.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data'
directories = ['processed_lsd_jpgs', 'processed_osf_jpgs', 'processed_spider_jpgs', 'processed_tseg_jpgs']

train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

# Create train and test directories if they don't exist
for d in directories:
    os.makedirs(os.path.join(train_dir, d), exist_ok=True)
    os.makedirs(os.path.join(test_dir, d), exist_ok=True)

# Iterate over each directory
for d in directories:
    image_dir = os.path.join(dataset_path, d)
    images = os.listdir(image_dir)

    # Split images into train and test
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Copy train images
    for img in train_images:
        src_path = os.path.join(image_dir, img)
        dest_path = os.path.join(train_dir, d, img)  # Save to train folder
        shutil.copy(src_path, dest_path)

    # Copy test images
    for img in test_images:
        src_path = os.path.join(image_dir, img)
        dest_path = os.path.join(test_dir, d, img)  # Save to test folder
        shutil.copy(src_path, dest_path)

print("Train-test split completed.")


Train-test split completed.


In [13]:
data_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data'


In [14]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [17]:
train_dir = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/train'
test_dir = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/test'

In [ ]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS):

  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  class_names = train_data.classes

  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )
  return train_datalaoder, test_dataloader, class_names


In [18]:
manual_transforms = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    normalize
]) # can also use automatic transforms using weights.transforms()

train_dataloaders, test_dataloaders, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=32
)

train_dataloaders, test_dataloaders, class_names

(<torch.utils.data.dataloader.DataLoader at 0x319bb0850>,
 ['processed_lsd_jpgs',
  'processed_osf_jpgs',
  'processed_spider_jpgs',
  'processed_tseg_jpgs'])

In [19]:
class_names

['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs']

In [20]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

In [21]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)


In [22]:
for param in model.features.parameters():
    param.requires_grad = False
    
# Since we're creating a new layer with random weights (torch.nn.Linear), 
# let's set the seeds
set_seeds() 

# Update the classifier head to suit our problem
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names),
              bias=True).to(device))

In [23]:
from torchinfo import summary


In [24]:
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 4]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    print("[INFO] Couldn't find tensorboard... installing it.")
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter


# Create a writer with all default settings
writer = SummaryWriter()

In [27]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
        
        # Track the PyTorch model architecture
        writer.add_graph(model=model, 
                         # Pass in an example input
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))
    
    # Close the writer
    writer.close()
    
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

In [29]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloaders,
                test_dataloader=test_dataloaders,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6684 | train_acc: 0.8233 | test_loss: 0.5110 | test_acc: 0.8620


 20%|██        | 1/5 [03:16<13:06, 196.51s/it]

Epoch: 2 | train_loss: 0.2865 | train_acc: 0.9171 | test_loss: 0.2999 | test_acc: 0.9102


 40%|████      | 2/5 [06:30<09:45, 195.18s/it]

...

 60%|██████    | 3/5 [09:46<06:30, 195.49s/it]

Epoch: 4 | train_loss: 0.1848 | train_acc: 0.9533 | test_loss: 0.1851 | test_acc: 0.9375


 80%|████████  | 4/5 [13:05<03:16, 196.66s/it]

Epoch: 5 | train_loss: 0.1654 | train_acc: 0.9485 | test_loss: 0.1683 | test_acc: 0.9336


100%|██████████| 5/5 [16:40<00:00, 200.11s/it]


In [30]:
results

{'train_loss': [0.6683949856988846,
  0.28645355446684745,
  0.2099980918630477,
  0.1847972752105805,
  0.16537261345694143],
 'train_acc': [0.8232758620689655,
  0.9171301446051168,
  0.9340586763070078,
  0.9533161846496108,
  0.948484427141268],
 'test_loss': [0.5109908431768417,
  0.2999418741092086,
  0.21004890464246273,
  0.1850558731239289,
  0.16827066976111382],
 'test_acc': [0.8619791666666666, 0.91015625, 0.9296875, 0.9375, 0.93359375]}

In [35]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 18714), started 0:01:06 ago. (Use '!kill 18714' to kill it.)

In [36]:
def create_writer(experiment_name: str, 
                  model_name: str, 
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
        
    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [39]:
# Create an example writer
example_writer = create_writer(experiment_name="data_100_percent",
                               model_name="effnetb0",
                               extra="3_epochs")

[INFO] Created SummaryWriter, saving to: runs/2024-09-16/data_100_percent/effnetb0/3_epochs...


In [40]:
from typing import Dict, List
from tqdm.auto import tqdm

# Add writer parameter to train()
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device, 
          writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss", 
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy", 
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc}, 
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

In [42]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloaders,
                test_dataloader=test_dataloaders,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=3,
                device=device,
                writer=example_writer)

  0%|          | 0/3 [03:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# making training set less ;)
